In [1423]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import label_binarize


# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import words
from nltk.corpus import brown
import pickle
import boto3

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

# Set execution role
client = boto3.client('s3') #low-level functional API
resource = boto3.resource('s3') #high-level object-oriented API
my_bucket = resource.Bucket('sagemaker-nomadiq-data') #subsitute this for your s3 bucket name. 


In [1424]:
with open('vocab_labels.pickle', 'wb') as f:
    pickle.dump([vocabulary,vocab_size,sentence_size,labelencoder,unique_labels,test_accuracy_list], f)

my_bucket.upload_file('vocab_labels.pickle',mybucket,'vocab_labels.pickle')

s3.ObjectSummary(bucket_name='sagemaker-nomadiq-data', key='IG_JSON.zip')
s3.ObjectSummary(bucket_name='sagemaker-nomadiq-data', key='full_data.csv')
s3.ObjectSummary(bucket_name='sagemaker-nomadiq-data', key='instagram_df.csv')
s3.ObjectSummary(bucket_name='sagemaker-nomadiq-data', key='loc_geo_map.csv')
s3.ObjectSummary(bucket_name='sagemaker-nomadiq-data', key='test_data.csv')
s3.ObjectSummary(bucket_name='sagemaker-nomadiq-data', key='testnet.params')
s3.ObjectSummary(bucket_name='sagemaker-nomadiq-data', key='training_data.csv')
s3.ObjectSummary(bucket_name='sagemaker-nomadiq-data', key='vocab_labels.pickle')
s3.ObjectSummary(bucket_name='sagemaker-nomadiq-data', key='wikitravel.csv')


# Get instagram scrape data

In [1323]:
# This data contains location id and location name mappings scraped from instagram
insta_df = pd.read_csv('instagram_df.csv')

In [1324]:
insta_df.head()

,caption,location_id,location_name,timestamp,hashtags,month,datetime_readable
0,"Hair kissed by the sun ☀\n••••••••••••••••••••••••••••••••••••••••••••••••\nTravel isn't always kind to my hair—especially in tropical destinations with lots of sun, sand, and saltwater! I started traveling with @Pantene's new Intense Rescue Shots (which are imported from Brazil and are available now at Pantene.com) to keep my hair healthy.\n\nUnlike other hair treatments, these shots provide deep conditioning, Pro-Vitamin B5, strengthening lipids and natural glycerin that targets the damaged areas in just 30 seconds, and rinses out quickly in 30 seconds to avoid weighed-down hair. Plus, their travel-friendly size means I can bring them on any trip—even in a carry-on! Check my LINK IN BIO for more info.\n••••••••••••••••••••••••••••••••••••••••••••••••\n#Ad #PlayRescueReset #PanteneRescueShots",NaN,NaN,1.548702e+09,"['ad', 'playrescuereset', 'pantenerescueshots']",1.0,1970-01-01 00:00:01.548701796
1,"One of my favorite views in the world 🗺 ••••••••••••••••••••••••••••••••••••••••••••••••\nIt’s amazing how much can happen in 24 hours. Yesterday was a perfect summer day in Cape Town, but today felt like the apocalypse. Earlier today, Lion’s Head mountain, one of the local natural landmarks, caught fire. Anyone that’s visited Cape Town knows just how hectic the winds can get and, with a wildfire, there’s no worse combination. The fire spread over to the edge of our neighborhood in a few short hours and the air is still thick with smoke and the sound of the wind is deafening. We’ve gotten an update that the local firefighters will be working through the night. Thankful for their courage and for keeping us safe 🙏 I’ve posted a couple clips in my stories with more info on the situation.\n••••••••••••••••••••••••••••••••••••••••••••••••\nShot on the @djiglobal #Mavic2Pro",215871574.0,"Cape Town, Western Cape",1.548620e+09,['mavic2pro'],1.0,1970-01-01 00:00:01.548620093
2,"Got my sights set on Sydney! 🐨🇦🇺 ••••••••••••••••••••••••••••••••••••••••••••••••\nI am so excited to be returning to Aus so soon and, this trip, I have time for a meetup! I’m planning a little Sunday Funday in Sydney on February 24th :) Click the link in bio or join “The Blonde Abroad Travel Tribe” group on FB to RSVP and get updated with all the details! Hope to meet some of you soon!\n••••••••••••••••••••••••••••••••••••••••••••••••\n#sydney #australia",2112249.0,Sydney Opera House,1.548445e+09,"['sydney', 'australia']",1.0,1970-01-01 00:00:01.548445014
3,"Planning a bucket list trip in 2019? #CapitalOnePartner ✈️ ••••••••••••••••••••••••••••••••••••••••••••••\nTravel reward credit cards have helped me earn points for ""free"" flights to some of my biggest bucket list destinations like The Galapagos, South Africa, Hong Kong and more. The @CapitalOne Venture card makes it easier than ever to earn Venture Miles on (literally) any purchase.\n\nPlus, with their new Miles Transfer benefit, you can transfer your Venture Miles directly to over a dozen top airlines partners to book reward flights, with just a few clicks!\n••••••••••••••••••••••••••••••••••••••••••••••\nI'm talking about the Capital One Venture Card Miles Transfer Benefit on the blog today! LINK IN BIO",NaN,NaN,1.548181e+09,['capitalonepartner'],1.0,1970-01-01 00:00:01.548180883
4,Back home in my favorite city ♥️🇿🇦\n••••••••••••••••••••••••••••••••••••••••••••••••\n#capetown #southafrica #tbapresets,215871574.0,"Cape Town, Western Cape",1.548102e+09,"['capetown', 'southafrica', 'tbapresets']",1.0,1970-01-01 00:00:01.548101892


# Get Wikitravel scrape data

In [1325]:
wiki_df = pd.read_csv('wikitravel.csv',sep = '\t', index_col= False)
# Remove entries with "errors"
wiki_df = wiki_df[(wiki_df['summary'] != 'error')|(wiki_df['do'] != 'error')|
                  (wiki_df['see'] != 'error')|(wiki_df['eat'] != 'error')]

# Map Locations to Location_IDs
Instagram IDs are often at the city + state + country level and the wiki travel data is at just the city level. We need to normalizes these to map to eachother.

In [1326]:
# First we zip the location_names and location_id from the instagram data
zipped_locations = list(zip(insta_df.location_name,insta_df.location_id))

In [1327]:
# Get first 5 rows of zipped data from instagram_data
zipped_locations[:5]

[(nan, nan),
 ('Cape Town, Western Cape', 215871574.0),
 ('Sydney Opera House', 2112249.0),
 (nan, nan),
 ('Cape Town, Western Cape', 215871574.0)]

In [1328]:
# We condense the large instagram scrape data by grouping to only unique combinations of location_name and location_id
insta_df2 = insta_df.groupby(['location_name','location_id']).size().reset_index().rename(columns={0:'count'})
zipped_locations = list(zip(insta_df2.location_name,insta_df2.location_id))

In [1329]:
# Get first 5 rows of zipped data from instagram_data
zipped_locations[:5]

[('"Beatles Ashram", Rishikesh', 237964411.0),
 ('"Greetings From Austin" Postcard Mural', 223569558.0),
 ('"La Maona - The original one!"', 296867213.0),
 ('"Los Angeles - City Of Dreams"', 252555922.0),
 ('"O" by Cirque du Soleil', 59018683.0)]

In [1330]:
# Compile list of cities from wiki scrape data
city_list = list(wiki_df.city)

In [1331]:
# Get first 5 rolws of city list
city_list[:5]

['A Coruña', 'Aachen', 'Aalborg', 'Aarhus', 'Abadan']

In [1332]:
# Create dictionary of city names and location ID using city list from wiki_travel and location ID mappings from instagram data.
# Output is a key value pair of city name and location id
city_dict = {}

# Loop through all cities in the wikitravel list of cities
for city in city_list:
    # Inner loop through all location_name/location_id mappings
    for location in zipped_locations:
        # If there is an exactly city match pull write the key/value
        # Else continue
        if city.lower() == location[0].lower():
            city_dict[city] = int(location[1])
            break
        else:
            continue
    # If no exact matches are found, we try soft matches where find the city name within the longer string in location_name
    try:
        city_dict[city]
    except:
        for location in zipped_locations:
            if city.lower() in location[0].lower():
                city_dict[city]= int(location[1])
                break
        else:
            continue
    

# Preprocess data for vectorization

In [1333]:
# Clean string with preprocessing regex rules
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
#     string = re.sub(r'\b\d+(?:\.\d+)?\s+', '', string)
#     string = re.sub(r"\d+", "", string)
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r"<b>", " ", string)
    string = re.sub(r"</b>", " ", string)
    string = re.sub(r"<br>", " ", string)
    string = re.sub(r"</br>", " ", string)
    string = re.sub(r"<p>", " ", string)
    string = re.sub(r"</p>", " ", string)
    string = re.sub(r"<ul>", " ", string)
    string = re.sub(r"</ul>", " ", string)
    string = re.sub(r"<li>", " ", string)
    string = re.sub(r"</li>", " ", string)  
    return string.strip().lower()


In [1334]:
# Concatenate all text from Wikitravel text into single column for each location
wiki_df['concat_text'] = wiki_df['summary'] + " " + wiki_df["do"] + " " + wiki_df['see'] + " " + wiki_df['eat']
wiki_df['concat_text'] = wiki_df['concat_text'].apply(clean_str)

In [1388]:
# Limit the wiki data to only those that are in the location_name/id mappings
wiki_df = wiki_df[wiki_df['city'].isin(list(city_dict.keys()))]
wiki_df.reset_index(drop = True, inplace = True)
# Dedupe the dataframe
wiki_df = wiki_df.drop_duplicates(['city'])
print("Number of Cities: ", len(wiki_df))
wiki_df.head()

Number of Cities:  1673


Unnamed: 0      city  \
0  0           A Coruña   
1  1           Aachen     
2  2           Aalborg    
3  3           Aarhus     
4  7           Abakan     

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               summary  \
0    \nA Coru\xc3\xb1a (Spanish: La Coru\xc3\xb1a) is located in Galicia, Spain.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
1    \nAachen [11] is a city in North Rhine-Westphalia, Germany, at the point where Germany borders on Belgium and the Netherlands. Historically this spa-town was a prominent city, the place where the German Kings were crowned, and a famously favoured residence of Charlemagne who still lies buried in the impressive cathedral he himself had built. The city is packed with historic sights that remind of those days, including medieval buildings, city gates and beautiful fountains.\nToday, Aachen has become a pleasant mix of historic (often Baroque) grandeur and modern innovation. It has a population of 246,000 and is home to a large and highly regarded university. All in all, a place well worth visiting.\nIn French (including crossborder train services from Brussels and Paris), the city is known as Aix-la-Chapelle, the city is called Aquisgr\xc3\xa1n in Spanish, whereas its Dutch name is Aken.   
2    \nAalborg [16] [17] is in North Jutland, Denmark. Its population, as of 2012, is 128,644, making it the fourth largest in Denmark.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
3      \n\nAarhus, (pronounced: Oar-Hoos, previously \xc3\x85rhus) is the main city on the Jutland peninsula in Denmark. With a population of just over 300,000 people (1,200,000 East Jutland metropolitan area) it als

# Build vocabulary and stopword list
The problem with using TFIDF for travel data is that location specific data will be heavily weighted. For example, an article about Tokyo will have the word "Toyko" appear many times. Tokyo is likely not included in many other articles except for others that are about Japan (e.g, Osaka, Kyoto, etc.). This would make the recommendation engine overly specific. Therefore, we limit the vocabulary to the english dictionary and remove location-specific strings.

In [1336]:
# Create unique location list and city list to include into stopwords
location_name_list = list(insta_df2.location_name.unique().astype(str))
location_name_list = [x.lower() for x in location_name_list]
cities = pd.read_csv('cities.csv', index_col= False)
city_list = list(cities['city'].astype(str)) + list(cities['country'].astype(str))
city_list = [x.lower() for x in city_list]
location_list = set(location_name_list + city_list)
location_list

{'kafu hostel & pool siem reap 薫風ホステル',
 'apam, central, ghana',
 'cathédrale la major',
 'medellín, antioquia, colombia',
 'georgeous coffee',
 'flateyri city',
 'saint-germain-en-laye, france',
 'the standard, high line',
 'bistro at edgewood',
 'queenstown',
 'costa dulce',
 'adelaide hills australia',
 'breenhold gardens',
 'shark valley visitor center',
 '恆春信用組合 credit union base',
 'marburg oberstadt',
 'sacred valley of the incas, peru',
 'knock sink woods',
 'monument valley, grand junction',
 'mesjid raya al-mahsun',
 'tazougart, khenchela, algeria',
 'the beach dubai jbr',
 'yibin',
 'karadeniz',
 'ægir bryggeri',
 'dartmouth, nova scotia',
 'zeitgeist coffee',
 'lexis suites penang',
 'chikmagalur',
 'college, alaska',
 'dcondo sign chiangmai',
 'ostia, lazio, italy',
 'thingvellir national park',
 'iberostar cancún',
 'tuk tuk indian street food',
 'hotel club amigo atlántico guardalavaca',
 '43°steakhouse 大通bisse',
 'princes st gardens',
 'the ritz-carlton, koh samui',
 't

In [1337]:
city_list2 = []
for city in city_list:
    split_city = city.split()
    for split in split_city:
        city_list2.append(split)

location_list = set(location_name_list + city_list2)

In [1338]:
# Concatenate NLTK words and brown corpuses for comprehensive vocabulary
english_vocab = set([w.lower() for w in words.words()] + [w.lower() for w in brown.words()])

In [1339]:
# Create Custom Stopwords unioning all english words and location-specific strings
my_stop_words = text.ENGLISH_STOP_WORDS.union(location_list)

# Vectorize text using TFIDF

In [1340]:
# We instantiate the TfidifVectorizer using our customer stopwords and limiting to the english vocab
vectorizer = TfidfVectorizer(stop_words = my_stop_words, vocabulary = english_vocab)
X_tfidf = vectorizer.fit_transform(wiki_df['concat_text'])

/Users/geneahn/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [1341]:
print("Number of Features:", len(vectorizer.get_feature_names()))

Number of Features: 261552


In [1342]:
# Get number of features
X_tfidf.shape

(1673, 261552)

# Determine Cosine Similarity
We use cosine similarity to determine pairwise similiarities of locations. 

In [1378]:
# We pass the TFIDF matrix to get pairwise similarities of all entries. 
cosine_sim = linear_kernel(X_tfidf, X_tfidf)
# example output of cosine similarity for our first city "A Coruna"
# There is a score for every city from 0 to 1 (1 means that the pair is identical).
cosine_sim[0]

array([1.        , 0.06265196, 0.06422977, ..., 0.1718409 , 0.07095382,
       0.06832169])

In [1427]:
#Construct a reverse map of cities to indices
indices = pd.Series(wiki_df.city)
reverse_indices = pd.Series(wiki_df.index, index=wiki_df['city'])

In [1428]:
indices.head()

0    A Coruña
1    Aachen  
2    Aalborg 
3    Aarhus  
4    Abakan  
Name: city, dtype: object

In [1433]:
# Pickle similarity score data, indices, wiki_df, tfidf_vectorizer, x_tfidf
with open('tfidf_artifacts.pickle', 'wb') as f:
    pickle.dump([cosine_sim,indices,reverse_indices,vectorizer,X_tfidf,city_dict], f)

my_bucket.upload_file('tfidf_artifacts.pickle','tfidf_artifacts.pickle')

In [1429]:
# Function that takes in city as input and outputs most similar cities
# Pickle sim_scores, indices, wiki_df, tfidf vectorizer, x_tfidf, clean_str?

def get_recommendations_city(city, cosine_sim=cosine_sim):
    city = city.title()
    idx = reverse_indices[city]
    # Get the pairwise similarity scores of all cities
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the cities based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the 10 most similar cities
    sim_scores = sim_scores[1:11]
    # Get the city indices
    city_indices = [i[0] for i in sim_scores]
    city_recs = []
    loop_count = 0
    for city in indices.iloc[city_indices]:
        city_recs.append({"location_id": city_dict[city], "location_name": city, "cosine_similarity": sim_scores[loop_count][1]})
        loop_count += 1
    # Return the top 10 most similar cities
    return json.dumps(city_recs)

In [1431]:
# Enter city name to get top 10 city names
get_recommendations_city('Seattle')

'[{"location_id": 214060412, "location_name": "Los Angeles", "cosine_similarity": 0.29995098914665747}, {"location_id": 223569558, "location_name": "Austin", "cosine_similarity": 0.29765384694601393}, {"location_id": 1165768, "location_name": "Buffalo", "cosine_similarity": 0.28275648023521743}, {"location_id": 613307678847655, "location_name": "Toronto", "cosine_similarity": 0.2797944668684217}, {"location_id": 226113440, "location_name": "Nashville", "cosine_similarity": 0.2742314336321626}, {"location_id": 44961364, "location_name": "San Francisco, California", "cosine_similarity": 0.2660621679660008}, {"location_id": 214228753, "location_name": "Philadelphia, Pennsylvania", "cosine_similarity": 0.2646629250299263}, {"location_id": 86478759, "location_name": "Omaha", "cosine_similarity": 0.2609433606657226}, {"location_id": 230222108, "location_name": "Houston", "cosine_similarity": 0.2605591341143785}, {"location_id": 213819997, "location_name": "Vancouver, British Columbia", "cosi

In [1412]:
# Get similar locations to a given keyword vector
doc = "scuba dive seafood"

In [1420]:
def get_recommendations_keywords(doc, cosine_sim=cosine_sim):
    test_tfidf = vectorizer.transform([clean_str(doc)])
    cosine_sim_test = linear_kernel(test_tfidf, X_tfidf)
    sim_scores = list(enumerate(cosine_sim_test[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[0:10]
    city_indices = [i[0] for i in sim_scores]
    city_recs = []
    loop_count = 0
    for city in wiki_df['city'].iloc[city_indices]:
        city_recs.append({"location_id": city_dict[city], "location_name": city, "cosine_similarity": sim_scores[loop_count][1]})
        loop_count += 1
    # Return the top 10 most similar cities
    return json.dumps(city_recs)

In [1421]:
get_recommendations_keywords("scuba dive seafood")

/Users/geneahn/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


'[{"location_id": 216849958, "location_name": "Kota Kinabalu", "cosine_similarity": 0.26078909705753134}, {"location_id": 213735962, "location_name": "Nha Trang", "cosine_similarity": 0.24719011078115571}, {"location_id": 217273353, "location_name": "Tawau", "cosine_similarity": 0.24043859036258564}, {"location_id": 783101726, "location_name": "Townsville", "cosine_similarity": 0.23461979199901029}, {"location_id": 285433299, "location_name": "Miri", "cosine_similarity": 0.23010974441992185}, {"location_id": 7607961, "location_name": "Banda", "cosine_similarity": 0.22428111785724245}, {"location_id": 282889198, "location_name": "Cairns", "cosine_similarity": 0.22203774386935676}, {"location_id": 274441659250894, "location_name": "Batam", "cosine_similarity": 0.19478269817329888}, {"location_id": 69094957, "location_name": "Cabo Frio", "cosine_similarity": 0.19003392298159882}, {"location_id": 214754776, "location_name": "Puerto Montt", "cosine_similarity": 0.17562641191753098}]'

In [1212]:
# you only needs to do this once, this is a mapping of index to 
features = vectorizer.get_feature_names()

# get the document that we want to 
# doc = df.concat_text[2101]
 
#generate tf-idf for the given document
tf_idf_vector = vectorizer.transform([doc])
 
#sort the tf-idf vectors by descending order of scores
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

sorted_items = sort_coo(tf_idf_vector.tocoo())
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(features,sorted_items,10)
 
# now print the results
print("\n=====Doc=====")
print(doc)
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Doc=====
scuba dive seafood

===Keywords===
dive 0.868
seafood 0.497


/Users/geneahn/anaconda/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [1426]:
input("hello:")

hello:asdf


'asdf'